### 01. Overview

#### 가설
* 가설1: 거주인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설2: 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설3: 유동인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
    * 역세권, 핫플레이스, 관광지
* 가설4: 대학교 근처에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설5: 신도시 주변에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설6: 인근 카페 수가 많을 수록 매장이 많이 입지해 있을 것이다.
* 가설7: 전국에서 서울시에 스타벅스 매장이 밀집되어 있을 것이다.
* 가설8: 직장군별 스타벅스 매장의 입지수의 차이가 있을 것이다.
* 가설9: 집값이 비싼 지역에 매장이 밀집되어 있을 것이다.

##### 데이터수집

* 1. 스타벅스 사이트에서 서울시에 존재하는 매장
    * 지점명,주소(+층수),매장별 소개 멘트,운영시간,매장종류(R,DT,..),주차정보,오시는 길 설명 (접근성),위도,경도 정보
    * https://www.starbucks.co.kr/index.do
* 2. 서울시 주민등록 인구(구별) 통계 정보 수집
    * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 3. 서울시 사업체 현황(사업대분류별/동별) 통계정보
    *https://data.seoul.go.kr/dataList/104/S/2/datasetView.do


##### 프로젝트 진행
* 가설 3개를 선택 후, 관련데이터 수집 
* 가설1(거주인구), 가설2(직장인), 가설3(타커피브랜드 입점위치)

### 02.데이터수집

#### 2-1. 서울시 스타벅스 매장목록

In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


# 드라이버 최신 버전 셋팅
service=Service()
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

url = "https://www.starbucks.co.kr/index.do"
driver.get(url)

In [8]:
## 추가 라이브러리 불러오기

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains 
import time

In [9]:

## 동적크롤링

actions=ActionChains(driver)

element = driver.find_element(By.CLASS_NAME,'gnb_nav03')
actions.move_to_element(element)
time.sleep(1)

element2 = driver.find_element(By.CSS_SELECTOR,\
    '#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li:nth-child(3) > a')
actions.click(element2)

actions.perform()


In [10]:
## 서울,전체 클릭하기

driver.find_element(By.CLASS_NAME,'set_sido_cd_btn').click()
time.sleep(1)
driver.find_element(By.CLASS_NAME,'set_gugun_cd_btn').click()


In [13]:
## 서울 스타벅스의 모든 데이터 가져오기

import requests
import bs4

res = driver.page_source
starbucks=bs4.BeautifulSoup(res,'html.parser')

In [14]:
print(type(starbucks))

<class 'bs4.BeautifulSoup'>


* 얻어야할 정보: 지점명,주소,오시는 길 설명 (접근성),위도,경도 정보
* 변수명: name, addr, way_to_come, lat, lng

In [15]:
## ul 가져오기 (모든 매장정보가 들어있는 list들)

ul=starbucks.select('li.quickResultLstCon')
print(len(ul))

611


In [26]:
## 서울시 스타벅스 매장 목록 데이터 만들기

starbucks_list = []

for startbucks_store in ul:
    name = startbucks_store.select('strong')[0].text.strip()
    lat = startbucks_store['data-lat'].strip()
    lng = startbucks_store['data-long'].strip()
    store_type = startbucks_store.select('i')[0]['class'][0]
    add_tel = str(startbucks_store.select('p.result_details')[0]).split('<br/>')
    address = add_tel[0].split('>')[1]
    tel = add_tel[1].split('<')[0]
    
    starbucks_list.append( [ name, lat, lng, store_type, address, tel])

In [30]:
## 데이터프레임 생성

import pandas as pd

columns = ['매장명','위도','경도','매장타입', '주소','전화번호']
seoul_starbucks_df = pd.DataFrame(starbucks_list, columns = columns)
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,pin_general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,pin_general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,pin_general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,pin_general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,pin_reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [31]:
## 요약 정보 확인

seoul_starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     611 non-null    object
 1   위도      611 non-null    object
 2   경도      611 non-null    object
 3   매장타입    611 non-null    object
 4   주소      611 non-null    object
 5   전화번호    611 non-null    object
dtypes: object(6)
memory usage: 28.8+ KB


In [35]:
## 엑셀로 저장

seoul_starbucks_df.to_excel('../files/seoul_starbucks(dhfjdfjdfh).xlsx', index=False)

### 2.2 서울시 통계 정보 수집: 공공데이터

#### 2-2-1. 서울시 주민등록 인구(구별) 데이터 분석

In [36]:
## txt파일 불러오기

import pandas as pd

sgg_pop_df = pd.read_csv('./report.txt',sep='\t', header=1)
sgg_pop_df.head()

,시점,동별(1),동별(2),소계,소계.1,남자,여자,소계.2,남자.1,여자.1,소계.3,남자.2,여자.2,소계.4,소계.5
0,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
1,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
2,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
3,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
4,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854


In [37]:
## 서울시 인구데이터 살펴보기

columns = {
    '시점': 'GIGAN',
    '동별(1)': 'SUM',
    '동별(2)': 'JACHIGU',
    '소계':'SEDAE',
    '소계.1': 'GYE_1',
    '남자':'NAMJA_1',
    '여자':'YEOJA_1',
    '소계.2': 'GYE_2', # 한국인
    '남자.1': 'NAMJA_2',
    '여자.1':'YEOJA_2',
    '소계.3': 'GYE_3', # 등록외국인
    '남자.2':'NAMJA_3',
    '여자.2':'YEOJA_3',
    '소계.4': 'SEDAEDANGINGU',
    '소계.5': 'N_65SEISANGGORYEONGJA'
}

sgg_pop_df.rename(columns=columns, inplace=True)
sgg_pop_df.head()

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
1,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
2,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
3,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
4,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854


In [38]:
## 요약정보 보기

sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  25 non-null     object 
 1   SUM                    25 non-null     object 
 2   JACHIGU                25 non-null     object 
 3   SEDAE                  25 non-null     int64  
 4   GYE_1                  25 non-null     int64  
 5   NAMJA_1                25 non-null     int64  
 6   YEOJA_1                25 non-null     int64  
 7   GYE_2                  25 non-null     int64  
 8   NAMJA_2                25 non-null     int64  
 9   YEOJA_2                25 non-null     int64  
 10  GYE_3                  25 non-null     int64  
 11  NAMJA_3                25 non-null     int64  
 12  YEOJA_3                25 non-null     int64  
 13  SEDAEDANGINGU          25 non-null     float64
 14  N_65SEISANGGORYEONGJA  25 non-null     int64  
dtypes: float

In [39]:
sgg_pop_df.head()

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
1,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
2,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
3,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
4,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854


In [40]:
## 필요없는 데이터 제거(임시 참고)

condition = (sgg_pop_df['JACHIGU'] != "소계")
sgg_pop_df_selected = sgg_pop_df[condition]
sgg_pop_df_selected.head()

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
1,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
2,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
3,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
4,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854


In [41]:
## 필요한 열

sgg_pop_df_selected[['JACHIGU',"GYE_1"]].head()

,JACHIGU,GYE_1
0,종로구,152212
1,중구,131390
2,용산구,232482
3,성동구,287240
4,광진구,350925


In [42]:
# 분석에 필요한 칼럼 선택
columns = ['JACHIGU','GYE_1']
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.head()

,JACHIGU,GYE_1
0,종로구,152212
1,중구,131390
2,용산구,232482
3,성동구,287240
4,광진구,350925


In [43]:
## 데이터프레임의 칼럼명 변경

sgg_pop_df_final.columns = ['시군구명', '주민등록인구']
sgg_pop_df_final.info()
sgg_pop_df_final.head()
# sgg_pop_df_final.rename(columns={'JACHIGU':'시군구명','GYE_1':'주민등록인구'})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구명    25 non-null     object
 1   주민등록인구  25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 532.0+ bytes


,시군구명,주민등록인구
0,종로구,152212
1,중구,131390
2,용산구,232482
3,성동구,287240
4,광진구,350925


In [44]:
## 엑셀로 저장

sgg_pop_df_final.to_excel('../files/sgg_pop.xlsx', index= False)

#### 2-2-2. 서울시 사업체현황(산업대분류별/동별) 데이터 분석

In [45]:
## txt파일 불러오기

sgg_biz_df = pd.read_csv('../files/report1.txt',sep='\t',header=3)
sgg_biz_df.head()

,시점,동별(1),동별(2),동별(3),소계,여성대표자,소계.1,남,여,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
1,2021,합계,종로구,사직동,5035,1708,56013,28597,27416,1,...,28,7047,132,927,120,1207,114,1612,285,944
2,2021,합계,종로구,삼청동,956,423,4978,2482,2496,-,...,3,1264,20,287,9,40,37,307,39,146
3,2021,합계,종로구,부암동,894,416,3762,1665,2097,-,...,3,65,55,1338,30,286,39,99,81,226
4,2021,합계,종로구,평창동,1321,577,4321,2068,2253,-,...,3,115,70,448,40,521,81,195,123,281


In [46]:
## 열이름 바꾸기

columns= {
    '시점': 'GIGAN',
    '동별(1)': 'GIGAN',
    '동별(2)': 'JACHIGU',
    '동별(3)': 'DONG',
    '소계': 'SAEOPCHESU_1',
    '여성대표자':'YEOSEONGDAEPYOJA',
    '소계.1':'GYE',
    '남':'NAM',
    '여':'YEO'
}
sgg_biz_df.rename(columns=columns, inplace=True)
sgg_biz_df.head()

,GIGAN,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
1,2021,합계,종로구,사직동,5035,1708,56013,28597,27416,1,...,28,7047,132,927,120,1207,114,1612,285,944
2,2021,합계,종로구,삼청동,956,423,4978,2482,2496,-,...,3,1264,20,287,9,40,37,307,39,146
3,2021,합계,종로구,부암동,894,416,3762,1665,2097,-,...,3,65,55,1338,30,286,39,99,81,226
4,2021,합계,종로구,평창동,1321,577,4321,2068,2253,-,...,3,115,70,448,40,521,81,195,123,281


In [47]:
## 시군구별 사업체 현황 데이터 추출(행)

condition =sgg_biz_df['DONG']=='소계'
sgg_biz_df_selected = sgg_biz_df[condition]
sgg_biz_df_selected.head(10)

,GIGAN,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
18,2021,합계,중구,소계,70308,25794,386564,213782,172782,1,...,65,16141,644,9425,813,11585,632,3258,2175,7632
34,2021,합계,용산구,소계,29680,10454,152605,84314,68291,4,...,45,3416,844,7352,632,8219,651,3006,1866,5572
51,2021,합계,성동구,소계,41665,13447,203221,116978,86243,7,...,45,5243,1283,10453,815,9635,844,2956,2341,7334
69,2021,합계,광진구,소계,33706,12297,129707,71124,58583,3,...,38,3592,1467,14698,981,12785,1139,3202,2799,5121
85,2021,합계,동대문구,소계,42813,14480,146383,78105,68278,2,...,49,4023,1288,16173,1085,17183,889,2282,2790,4906
100,2021,합계,중랑구,소계,39310,13471,113144,59221,53923,-,...,47,4108,1151,7668,1111,14806,1029,2273,3099,4937
117,2021,합계,성북구,소계,34712,13393,124004,60275,63729,1,...,49,4607,1806,21226,1131,16668,1043,2444,2931,5502
138,2021,합계,강북구,소계,26385,9807,80222,38440,41782,1,...,42,3530,890,6164,867,11540,869,1885,2508,4270
152,2021,합계,도봉구,소계,25391,8886,79097,41600,37497,2,...,37,4256,1114,7191,859,12804,738,1594,2264,4184


In [48]:
## 필요없는 칼럼 제거(열) 및 열이름 셋팅

columns=['JACHIGU','GYE','SAEOPCHESU_1']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns=['시군구명','종사자수','사업체수']

In [49]:
## 최종데이터

sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,275063,48361
18,중구,386564,70308
34,용산구,152605,29680
51,성동구,203221,41665
69,광진구,129707,33706


In [50]:
## 데이터프레임의 인덱스 초기화

sgg_biz_df_final =sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,275063,48361
1,중구,386564,70308
2,용산구,152605,29680
3,성동구,203221,41665
4,광진구,129707,33706


In [51]:
## 엑셀로 저장

sgg_biz_df_final.to_excel('../files/sgg_biz.xlsx', index=False)